In [1]:
import numpy as np
import pandas as pd
import seaborn as sns;sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import statsmodels.api as sm
import math
from math import sqrt

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer

from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix, classification_report, auc
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold

from sklearn import tree
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from IPython.display import Image

from sklearn import neighbors
import itertools
import plotly.express as px

from itertools import chain
from matplotlib.pyplot import figure

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import svm

from pprint import pprint
from time import time
import logging

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from collections import defaultdict
#nltk.download('all')
#nltk.download('averaged_perceptron_tagger')

from collections import Counter
import re
import string

from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS

from textblob import TextBlob

from wordhoard import Antonyms
import random

from sklearn.pipeline import Pipeline

pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [73]:
df=pd.read_csv('train.txt',delimiter=";",header=None,names=['corpus','feeling'])

In [108]:
df.head()

,corpus,feeling
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


### Data Cleaning

In [75]:
# Agrego un diccionario con contracciones y sus extensiones respectivas
contractions = { 
"ain't": "am not",
"aren't": "am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [76]:
#Agrego las mismas palabras de las stop words pero sin apostrofes (ej: haven't, agrego havent)
stop_words=stopwords.words('english');

hifen =' '.join(stop_words)
hifen = re.sub("'", '', hifen)    
hifen=word_tokenize(hifen)

stop_words.extend(hifen)
stop_words.append('shall')
stop_words.append('im')

In [77]:
clases=list(df.feeling.unique())
clases=sorted(clases)
clases

['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']

## Entrenamiento de modelos

### Dataset de Test

In [78]:
df_test=pd.read_csv('test.txt',delimiter=";",header=None,names=['corpus','feeling'])

In [79]:
df_test.head()

,corpus,feeling
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


### Modelo MultinomialNB con df_clean

In [104]:
class negation_transformer2(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    @staticmethod
    def negate_sequence(text):
        
        from wordhoard import Antonyms
        import random

        negation = False
        delims = "?.,!:;"
        result = []
        words = text.split()
        prev = None
        pprev = None
        for word in words:
            stripped = word.strip(delims).lower()
            negated = "not " + stripped if negation else stripped
            result.append(negated)

            if prev:
                bigram = prev + " " + negated
                pprev = prev
            prev = negated

            if any(neg in word for neg in ["not", "n't", "no"]):
                negation = not negation

            if any(c in word for c in delims):
                negation = False

        for elements in result:

            element=word_tokenize(elements)

            element0=0
            element_change=1

            if element[0]=="not" and len(element)>1 and element[1]!="feel":
                
                element0=" ".join(element)
                
                element_change=random.choice(Antonyms(search_string=element[1], output_format='list',max_number_of_requests=10,rate_limit_timeout_period=10).find_antonyms())
        
            result=[element_change if item == element0 else item for item in result]
                    
            '''        
            if element[0]=="not" and len(element)>1 and element[1]=="feel":
                i=result.index(elements)
                if i+1==len(result):
                    result= result[:i]+[element[1]]
                elif i+1<len(result):
                    result= result[:i]+[element[1]]+result[i+1:]

            if element[0]=="not" and len(element)==1:
                i=result.index(elements)
                if i+1==len(result):
                    result= result[:i]+[""]
                elif i+1<len(result):
                    result= result[:i]+[""]+result[i+1:]
            '''

        return " ".join(result)
        
    def fit(self, text, y=None):
         return self

    def transform(self, texto):
        if isinstance(texto, pd.Series):
            texto=texto.apply(lambda x: self.negate_sequence(x))
        else:
            texto=pd.Series(texto)
            texto=texto.apply(lambda x: self.negate_sequence(x))
            
        return texto

In [105]:
class clean_texto(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    @staticmethod
    def clean_text(text):
        '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''

        # Convert words to lower case
        text = text.lower()

        # Replace contractions with their longer forms 
        if True:
            text = text.split()
            new_text = []
            for word in text:
                if word in contractions:
                    new_text.append(contractions[word])
                else:
                    new_text.append(word)
            text = " ".join(new_text)

        # Format words and remove unwanted characters
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)

        # remove stop words
        #if remove_stopwords:
        text = text.split()
        stops = stop_words
        text = [w for w in text if not w in stops]
        text = " ".join(text)

        # spelling correction (tarda demasiado y no da buenos resultados)
        #text = TextBlob(text).correct() 
        #text = text.__str__() # opcion 1 (tarda demasiado)
        #text = '{}'.format(text) #opcion2 (tarda demasiado)
        #text = str(TextBlob(text).correct()) #opcion3 (tarda demasiado)

        #Lemmatize 

        #Defaultdict is a container like dictionaries present in the module collections. 
        #Defaultdict is a sub-class of the dictionary class that returns a dictionary-like object. 
        #The functionality of both dictionaries and defaultdict are almost same except for the fact that defaultdict never raises a KeyError. 
        #It provides a default value for the key that does not exists.
        
        tag_map = defaultdict(lambda : wn.NOUN)
        tag_map['J'] = wn.ADJ
        tag_map['V'] = wn.VERB
        tag_map['R'] = wn.ADV

        tokens = word_tokenize(text)
        lmtzr = WordNetLemmatizer()

        new_text2=[]

        for token, tag in pos_tag(tokens):
            lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
            new_text2.append(lemma)
            text=" ".join(new_text2)
        
        return text
    
    def fit(self, texto, y=None):
        return self

    def transform(self, texto):
        if isinstance(texto, pd.Series):
            texto=texto.apply(lambda x: clean_texto.clean_text(x))
        else:
            texto=pd.Series(texto)
            texto=texto.apply(lambda x: clean_texto.clean_text(x))
            
        return texto

In [106]:
parameters_vect = {
   # "max_df": (0.5),
    "strip_accents": ('unicode'), 
    "stop_words": (None),
    'max_features': (5000),
    "ngram_range": (1, 2),  
    'use_idf': (True),
    'sublinear_tf': (True),
    'norm': ('l1')
}

parameters_clf ={
    "alpha": (0.5),
    "fit_prior": (False)
}


pasos = [('negation_transformer',negation_transformer2()),
         ('clean_texto',clean_texto()),
         ("vect", TfidfVectorizer(**parameters_vect)),
         ("clf", MultinomialNB(**parameters_clf))]

pipe=Pipeline(pasos)

In [107]:
pipe.fit(df.corpus, df.feeling)

TypeError: _backoff_handler() takes 1 positional argument but 2 were given

In [ ]:
X_test_mnb=df_test['corpus']

In [ ]:
y_test_mnb=df_test['feeling']

In [ ]:
# Calculamos las predicciones y métricas sobre la data de prueba

predicciones=pipe.predict(X_test_mnb)
sns.heatmap(confusion_matrix(y_test_mnb, predicciones), annot=True, fmt='4d',xticklabels=clases, yticklabels=clases)
plt.xlabel('Predichos')
plt.ylabel('Reales')
plt.show()
print(classification_report(y_test_mnb, predicciones, digits=4))

In [ ]:
x=pipe.predict("Today I feel confident while talking to people, and feel that I have been able to develop some control over my own life.")
x[0]

## Negation Handling Test

Esta función SI FUNCIONA pero no puedo inculirla en el Pipeline

In [90]:
 def negate_sequence(text):

        negation = False
        delims = "?.,!:;"
        result = []
        words = text.split()
        prev = None
        pprev = None
        for word in words:
            stripped = word.strip(delims).lower()
            negated = "not " + stripped if negation else stripped
            result.append(negated)

            if prev:
                bigram = prev + " " + negated
                pprev = prev
            prev = negated

            if any(neg in word for neg in ["not", "n't", "no"]):
                negation = not negation

            if any(c in word for c in delims):
                negation = False

        for elements in result:

            element=word_tokenize(elements)
            
            element0=0
            element_change=1

            if element[0]=="not" and len(element)>1 and element[1]!="feel":
                
                element0=" ".join(element)
                
                element_change=random.choice(Antonyms(search_string=element[1], output_format='list',max_number_of_requests=10,rate_limit_timeout_period=10).find_antonyms())
        
            result=[element_change if item == element0 else item for item in result]

        return " ".join(result)

In [91]:
negate_sequence("I don't feel sad")

"i don't not feel up"